### Scaling - Cedric Liang

#### Image Selection

Now that our science images have been processed according to the procedure described in the opening chapters of Rieke, we must process the images further to remove problematic systemic issues in our data.

A great deal of our images were taken in imperfect conditions without the usage of the auto-tracker. As such, there is noticeable movement of the open cluster in our images, and we must computationally correct for these imperfections.

However, it first makes sense to get a statistical overview of the images and use some common sense to discard the ones that are clearly unfit for use. There was intermittent cloud cover on the night of our observations, and as such a reasonable portion (30-40%) of the images were taken such that the background was significantly elevated due to the reflection of terrestrial light - in some cases up to 20,000 counts.

As such, it makes sense for us to first analyse our data and determine which images to simply discard.

There is also a human element to this - it is more difficult for computational methods to discard qualitatively poor images that can impact our results, such as images with poor focus or streaking. Indeed, we noticed on the night of observation that some images had trails. With the quantity of data that we have, it makes more sense to discard these manually.


In [1]:
import numpy as np
import astropy
import ccdproc
from ccdproc import CCDData, Combiner
from astropy import units as u
from astropy.visualization import SqrtStretch
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm
from photutils.centroids import centroid_com, centroid_1dg, centroid_2dg
from photutils import CircularAperture
from photutils import aperture_photometry
from photutils import Background2D
from photutils import MedianBackground
from scipy.ndimage import shift
import gc                                                           
gc.enable()

import warnings
import os
warnings.filterwarnings('ignore')
ROOT_PATH = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# import the statistics printing methods from the flats notebook
from ipynb.fs.defs.flats import image_stats, print_stats

#key values for our bands
BANDS = ["V", "B", "R"]

ModuleNotFoundError: No module named 'ipynb'

In [10]:
proc_files = {band: ccdproc.ImageFileCollection(
    f"{ROOT_PATH}/src/processed_ims/", glob_include=f"proc_NGC_3293_{band}*") for band in BANDS}

scim = {
    band: [CCDData.read(f"{ROOT_PATH}/src/processed_ims/{fn}")
           for fn in image_files.files_filtered(PICTTYPE = 1)]
    for band, image_files
    in proc_files.items()}

Let's check one thing first - we took exposures of varying exposure times on the night of observation. As such, we'll need to take those into account when thinking about combining them.

In [33]:
from collections import Counter

for band in BANDS:
    print(f"Exposure time occurrences in {band} band: ", Counter(image.header['EXPTIME'] for image in scim[band]))

Exposure time occurrences in V band:  Counter({60.0: 20, 5.0: 10})
Exposure time occurrences in B band:  Counter({5.0: 20, 60.0: 20})
Exposure time occurrences in R band:  Counter({60.0: 20, 5.0: 10})


In [32]:
# print(scim["V"][0])
# print_stats("First V band science image stats: ", image_stats(scim["V"][0]))
# print_stats("Last V band science image stats: ", image_stats(scim["V"][-1]))

# print("\nMean pixel counts for each Vband image: \n", [float(stat_dict['Mean']) for stat_dict in map(image_stats, scim["V"])])


I started with the V-band and viewed the images in ds9 to visually inspect these images for tracking and focussing errors.

V-band images:
Looks good: 3862-3871, 3922, 3923, 3925, 3926, 3927, 4052, 4062, 4064, 4098, 4099
Slight tracking errors: 3929, 4053, 4063
Big tracking errors: 3920, 3921, 3924, 3928, 4065
Lots of cloud: 4058, 4059